## Letterboxd Data exploration

The Data used in this project can be found here: https://www.kaggle.com/datasets/gsimonx37/letterboxd
It will be placed in the directory "letterboxd", that is added to the .gitignore because of file size limitations in GitHub.

In [ ]:
print("test")